# Amendments 

### Requirements
```
python                    3.6
cobra                     0.13.4
memote                    0.8.6
jupyter                   1.0.0
Escher                    1.6.0
```

In [1]:
from cobra.io import read_sbml_model,write_sbml_model
from os import getcwd
import warnings

In [2]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    mcaps = read_sbml_model('../ModelFiles/xml/iMcBath.xml')

In [3]:
from cobra import Reaction, Metabolite

Reopen previously constrained reactions (as a consequence of discovering that without this no growth on CO2 + H2 was possible)

In [4]:
mcaps.reactions.PGK.bounds = -1000, 1000
mcaps.reactions.GAPD.bounds = -1000, 1000
mcaps.reactions.GAPDH_nadp.bounds = -1000, 1000

## Growth on Amino Acids as Nitrogen Source:

In accordance with the low-copper growth conditions used by Patel et al [1], the sMMO is active. In absence of extracellular NH3 both GS/GOGAT and ADH operate according to [2], hence the reactions GLNS and ALAD_L were constrained to allow flux.

[1] Patel, R. N., and Hoare, D. S. (1971). Oxidation of C-1 Compounds capsulatus Physiological Studies of Methane and Methanol- Oxidizing Bacteria : Oxidation of C-1 Compounds by Methylococcus caps ulatus. J. Bacteriol. 107, 187–192.

[2] Murrell, J. C., and Dalton, H. (1983). Ammonia Assimilation in Methylococcus-Capsulatus (Bath) and Other Obligate Methanotrophs. J. Gen. Microbiol. 129, 1197–1206. doi:10.1099/00221287-129-4-1197.

In [5]:
# Switch medium to NO3, and sMMO utilization
mcaps.reactions.EX_n2_e.bounds = 0, 0
mcaps.reactions.PMMOipp.bounds = 0, 0
mcaps.reactions.PMMODCipp.bounds = 0, 0
mcaps.reactions.CYOR_q8ppi.bounds = 0, 1000
mcaps.reactions.EX_no3_e.bounds = -1000, 1000
mcaps.reactions.SMMOi.bounds = 0, 1000
mcaps.reactions.GLNS.bounds = 0, 1000
mcaps.reactions.ALAD_L.bounds = 0, 1000

In [6]:
from math import log

def dt2gr(dt):
    print (log(2)/ dt)

In [7]:
# Doubling times measured by [1] converted into growth rates

# nit
dt2gr(6)
# amm
dt2gr(8)
# l_ala
dt2gr(9)
# l_asp
dt2gr(6.5)
# l_asn
dt2gr(6.5)
# l_cys
dt2gr(8)
# l_glu
dt2gr(6.5)
# l_gln
dt2gr(6.5)
# l_val
dt2gr(9.5)

0.11552453009332421
0.08664339756999316
0.07701635339554948
0.10663802777845312
0.10663802777845312
0.08664339756999316
0.10663802777845312
0.10663802777845312
0.07296286111157319


Implement Mercaptopyruvate sulfurtransferase to allow the conversion of cysteine into pyruvate via mercaptopyruvate.

In [8]:
cyan = Metabolite(id='cyan_c', name='Hydrogen cyanide', charge=0, compartment='c', formula='CHN')
cyan.annotation = {'kegg.compound': ["C00177", "C01326", "C18673"],
"metanetx.chemical": "MNXM254",
"seed.compound": ["cpd00150", "cpd19012"],
"biocyc": ["META:CPD-13584", "META:HCN"],
"chebi": ["CHEBI:13362", "CHEBI:14038", "CHEBI:17514", "CHEBI:18407", "CHEBI:36856", "CHEBI:3969", "CHEBI:41780", "CHEBI:5786"],
"hmdb": "HMDB60292"}

In [9]:
tcynt = Metabolite(id='tcynt_c', name='Thiocyanate', charge=-1, compartment='c', formula='CNS')
cyan.annotation = {'kegg.compound': "C01755",
"metanetx.chemical": "MNXM762",
"seed.compound": "cpd01211",
"biocyc": "META:CPD-19512",
"chebi": ["CHEBI:15234", "CHEBI:18022", "CHEBI:24926", "CHEBI:24928", "CHEBI:26954", "CHEBI:26956", "CHEBI:29200", "CHEBI:45576", "CHEBI:9550"],
"hmdb": "HMDB01453"}

In [10]:
rxn = Reaction('MCPST')
rxn.add_metabolites({mcaps.metabolites.mercppyr_c: -1,
                    cyan: -1,
                    mcaps.metabolites.h_c: 1,
                    mcaps.metabolites.pyr_c: 1,
                    tcynt: 1},combine=False)
rxn.name = '3-mercaptopyruvate sulfurtransferase'
rxn.id = 'MCPST'
rxn.annotation = {
 u'biocyc': u'META:MERCAPYSTRANS-RXN',
 u'brenda': u'2.8.1.2',
 u'kegg.reaction': u'R03106',
 u'metanetx.reaction': u'MNXR101422'}
rxn.gene_reaction_rule = 'MCA1555'

mcaps.add_reaction(rxn)

In [11]:
mcaps.add_boundary(metabolite=mcaps.metabolites.cyan_c, type='sink')
mcaps.add_boundary(metabolite=mcaps.metabolites.tcynt_c, type='demand')

Reaction identifier,DM_tcynt_c
Name,Thiocyanate demand
Memory address,0x0113d4b860
Stoichiometry,tcynt_c --> Thiocyanate -->
GPR,
Lower bound,0
Upper bound,1000.0


Uptake of Amino Acids is modelled using sink reactions when there is insufficient information about transport reaction energetics. This may affect the predicted growth rates.

The accumulation of keto-acids is modelled using demand reactions when there is insufficient information about corresponding transporters. Again, this will also affect the predicted growth rates.

In [12]:
mcaps.add_boundary(metabolite=mcaps.metabolites.ala__L_c, type='sink')
mcaps.reactions.SK_ala__L_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.asn__L_c, type='sink')
mcaps.reactions.SK_asn__L_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.cys__L_c, type='sink')
mcaps.reactions.SK_cys__L_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.glu__L_c, type='sink')
mcaps.reactions.SK_glu__L_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.gln__L_c, type='sink')
mcaps.reactions.SK_gln__L_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.val__L_c, type='sink')
mcaps.reactions.SK_val__L_c.bounds = 0, 0

mcaps.add_boundary(metabolite=mcaps.metabolites.pyr_c, type='demand')
mcaps.add_boundary(metabolite=mcaps.metabolites.oaa_c, type='demand')
mcaps.add_boundary(metabolite=mcaps.metabolites.akg_c, type='demand')
mcaps.add_boundary(metabolite=mcaps.metabolites.get_by_id("3mob_c"), type='demand')

Reaction identifier,DM_3mob_c
Name,3-methyl-2-oxobutanoate demand
Memory address,0x0113c0b390
Stoichiometry,3mob_c --> 3-methyl-2-oxobutanoate -->
GPR,
Lower bound,0
Upper bound,1000.0


Investigate growth on experimentally validated amino acids as Nitrogen-sources:

In [13]:
from operator import attrgetter
from numpy import zeros
from six import iteritems
import pandas as pd

In [43]:
def summary(model, solution):
    metabolites = {m for r in model.boundary for m in r.metabolites}
    index = sorted(metabolites, key=attrgetter('id'))
    metabolite_fluxes = pd.DataFrame({'id': [m for m in index],'flux': zeros(len(index), dtype=float)}, index=[m.id for m in index])
    for rxn in model.boundary:
        for met, stoich in iteritems(rxn.metabolites):
            metabolite_fluxes.at[met.id, 'flux'] += stoich * solution[rxn.id]
    print ("IN")
    print ("----------")
    for x,y in metabolite_fluxes["flux"].items():
        if round(y, 4) > 0:
            print (x,round(y, 4))
    print ("OUT")
    print ("----------")
    for x,y in metabolite_fluxes["flux"].items():
        if round(y, 4) < 0:
            print (x,round(-y, 4))
    print ("Growth Rate")
    print ("----------")
    print (solution.fluxes.BIOMASS_Mcapsulatus)

In [15]:
def open_bounds():
    for met in mcaps.metabolites:
        if met.id not in ['na1_c', 'ca2_c', 'h_c', 'cl_c']:
            rxn_id = "TEST_" + met.id
            rxn = mcaps.add_boundary(met, reaction_id=rxn_id, type="custom", ub=1000, lb=0)

In [16]:
from escher import Builder

def visualize_fluxes(model, solution):
    flux_map = Builder(map_json="../MetabolicMap/MethylococcusCapsulatusMetabolismMap.json", model=model, reaction_data=solution.fluxes.to_dict())
    flux_map.display_in_browser()

In [17]:
from cobra.flux_analysis import pfba

In [54]:
# NO3 - Reference

with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -10, 0
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)
    summary(mcaps, solution)
    #visualize_fluxes(mcaps, solution)

IN
----------
ca2_e 0.0124
ch4_e 18.46
cl_e 0.0382
cu2_e 0.0003
fe2_e 0.0011
h_e 1.4584
k_e 0.0314
mg2_e 0.022
na1_e 0.007
no3_e 1.4587
o2_e 27.807
pi_e 0.0981
so4_e 0.03
zn2_e 0.0002
OUT
----------
co2_e 12.4681
doxopa_c 0.0002
h2o2_c 0.0572
h2o_e 32.9606
rdmbzi_c 0.0002
Growth Rate
----------
0.17800418171840207


In [55]:
# NH4 - Reference

with mcaps:
    mcaps.reactions.EX_nh4_e.bounds = -10, 0
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.GLNS.bounds = 0, 0
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    solution = pfba(mcaps)
    summary(mcaps, solution)
    #visualize_fluxes(mcaps, solution)

IN
----------
ca2_e 0.0182
ch4_e 18.46
cl_e 0.0558
cu2_e 0.0004
fe2_e 0.0016
k_e 0.0459
mg2_e 0.0321
na1_e 0.0102
nh4_e 2.1328
o2_e 27.861
pi_e 0.1435
so4_e 0.0439
zn2_e 0.0003
OUT
----------
co2_e 9.699
doxopa_c 0.0003
h2o2_c 0.0836
h2o_e 33.2636
h_e 2.1331
rdmbzi_c 0.0003
Growth Rate
----------
0.2602661442087391


In [47]:
# L-Alanine 

with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SK_ala__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
#     visualize_fluxes(mcaps, solution)

IN
----------
ala__L_c 10.0
ca2_e 0.0702
ch4_e 18.46
cl_e 0.2153
cobalt2_e 0.0001
cu2_e 0.0015
fe2_e 0.0061
h_e 1.7689
k_e 0.1772
mg2_e 0.124
na1_e 0.0393
o2_e 31.9631
pi_e 0.5537
so4_e 0.1695
zn2_e 0.001
OUT
----------
co2_e 14.6531
doxopa_c 0.0011
h2o2_c 0.3226
h2o_e 42.8106
nh4_e 1.7701
rdmbzi_c 0.0011
Growth Rate
----------
1.0043169244497043


In [20]:
# L-Aspartic acid
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.EX_asp__L_e.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
asp__L_e 10.0
ca2_e 0.07
ch4_e 18.46
cl_e 0.2149
cobalt2_e 0.0001
cu2_e 0.0015
fe2_e 0.0061
h_e 11.7837
k_e 0.1769
mg2_e 0.1237
na1_e 0.0392
o2_e 32.0259
pi_e 0.5527
so4_e 0.1692
zn2_e 0.001
OUT
----------
co2_e 24.7138
h2o_e 42.836
nh4_e 1.7849
Growth Rate
----------
1.0025128279064115


In [48]:
# L-Asparagine
with mcaps:
    open_bounds()
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SK_asn__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
asn__L_c 10.0
ca2_e 0.0721
ch4_e 18.46
cl_e 0.2212
cobalt2_e 0.0001
cu2_e 0.0016
fe2_e 0.0063
h_e 11.5426
k_e 0.1821
mg2_e 0.1273
na1_e 0.0404
o2_e 31.0114
pi_e 0.5688
so4_e 0.1741
zn2_e 0.001
OUT
----------
co2_c 23.7316
doxopa_c 0.0011
h2o2_c 0.3314
h2o_c 32.4276
nh4_c 11.5439
rdmbzi_c 0.0011
urea_c 0.0011
Growth Rate
----------
1.0316607036217855


In [49]:
# L-Cysteine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_cys__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)

IN
----------
ca2_e 0.0698
ch4_e 18.46
cl_e 0.2141
cobalt2_e 0.0001
cu2_e 0.0015
cyan_c 9.8314
cys__L_c 10.0
fe2_e 0.0061
k_e 0.1763
mg2_e 0.1233
na1_e 0.0391
o2_e 32.409
pi_e 0.5507
zn2_e 0.001
OUT
----------
co2_e 14.8397
doxopa_c 0.0011
h2o2_c 0.3209
h2o_e 42.7199
h_e 8.3542
nh4_e 1.8155
rdmbzi_c 0.0011
tcynt_c 9.8314
Growth Rate
----------
0.9987733659763532


In [50]:
# L-Glutamic acid
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_glu__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
ca2_e 0.0325
ch4_e 18.46
cl_e 0.0998
cu2_e 0.0007
fe2_e 0.0028
glu__L_c 10.0
k_e 0.0822
mg2_e 0.0575
na1_e 0.0182
o2_e 27.996
pi_e 0.2567
so4_e 0.0786
zn2_e 0.0005
OUT
----------
akg_c 9.4291
co2_e 5.64
doxopa_c 0.0005
h2o2_c 0.1496
h2o_e 32.0909
h_e 2.6745
nh4_e 6.1843
rdmbzi_c 0.0005
Growth Rate
----------
0.465644740246284


In [51]:
# L-Glutamine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_gln__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    
    summary(mcaps, solution)

IN
----------
ca2_e 0.0325
ch4_e 18.46
cl_e 0.0998
cu2_e 0.0007
fe2_e 0.0028
gln__L_c 10.0
k_e 0.0822
mg2_e 0.0575
na1_e 0.0182
o2_e 27.996
pi_e 0.2567
so4_e 0.0786
zn2_e 0.0005
OUT
----------
akg_c 9.4291
co2_e 5.64
doxopa_c 0.0005
h2o2_c 0.1496
h2o_e 22.0909
h_e 2.6745
nh4_e 16.1843
rdmbzi_c 0.0005
Growth Rate
----------
0.4656447402462855


In [52]:
# L-Valine
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.SK_val__L_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)

IN
----------
ca2_e 0.0319
ch4_e 18.46
cl_e 0.0978
cu2_e 0.0007
fe2_e 0.0028
k_e 0.0805
mg2_e 0.0563
na1_e 0.0179
o2_e 27.9897
pi_e 0.2515
so4_e 0.077
val__L_c 10.0
zn2_e 0.0005
OUT
----------
3mob_c 9.6464
co2_e 4.8751
doxopa_c 0.0005
h2o2_c 0.1465
h2o_e 31.9269
h_e 3.3844
nh4_e 6.2625
rdmbzi_c 0.0005
Growth Rate
----------
0.4560979557095218


Growth is predicted using any of the listed Amino Acids as Nitrogen sources, however, carbon acid excretion is only predicted for Valine, Glutamate and Glutamine. Presumably this is because pyruvate and oxaloacetate can be oxidized completely to CO2. This also explains the comparatively high CO2 production in the model constrained to uptake 

# Growth on H2 and CO2, as sources of energy and carbon respectively

According to Baxter et al. [1], M. capsulatus is able to grow autotrophically on H2 and CO2 or Formate and CO2. With the former as a source of energy and the latter as a source for carbon.

[1] Baxter, N. J., Hirt, R. P., Bodrossy, L., Kovacs, K. L., Embley, T. M., Prosser, J. I., et al. (2002). The ribulose-1,5-bisphosphate carboxylase/oxygenase gene cluster of Methylococcus capsulatus (Bath). Arch. Microbiol. 177, 279–289. doi:10.1007/s00203-001-0387-x.

In [26]:
# Add a dedicated sink reactions for H2 and formate
mcaps.add_boundary(metabolite=mcaps.metabolites.h2_c, type='sink')
mcaps.reactions.SK_h2_c.bounds = 0, 0
mcaps.add_boundary(metabolite=mcaps.metabolites.for_c, type='sink')
mcaps.reactions.SK_for_c.bounds = 0, 0

In [27]:
# H2 + CO2
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.EX_ch4_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = -18.46, 0
    mcaps.reactions.SK_h2_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
    

IN
----------
ca2_e 0.0037
cl_e 0.0114
co2_e 1.7871
cu2_e 0.0001
fe2_e 0.0003
h_e 0.435
k_e 0.0094
mg2_e 0.0066
na1_e 0.0021
no3_e 0.435
o2_e 2.282
pi_e 0.0293
so4_e 0.009
zn2_e 0.0001
OUT
----------
h2o_e 8.8191
Growth Rate
----------
0.05309000103804224


In [56]:
# H2
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.EX_ch4_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = 0, 0
    mcaps.reactions.SK_h2_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
    

IN
----------
h2_c 3.7289
o2_e 1.8644
OUT
----------
h2o_e 3.7289
Growth Rate
----------
1.8362663351804603e-16


In [61]:
# CO2
with mcaps:
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.EX_ch4_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = -18.46, 0
    mcaps.reactions.SK_h2_c.bounds = 0, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
    

Infeasible: None (infeasible)

In [28]:
# Formate + CO2
with mcaps:
    open_bounds()
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.EX_ch4_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = -18.46, 0
    mcaps.reactions.SK_for_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
    
    # visualize_fluxes(mcaps, solution)
    

There is an `e` compartment but it does not look like it is the actual external compartment.
There is an `e` compartment but it does not look like it is the actual external compartment.


IN
----------
ca2_e 0.0043
cl_e 0.0131
cu2_e 0.0001
fe2_e 0.0004
h_e 10.5021
k_e 0.0108
mg2_e 0.0076
na1_e 0.0024
no3_e 0.5022
o2_e 1.8628
pi_e 0.0338
so4_e 0.0103
zn2_e 0.0001
OUT
----------
co2_c 7.9372
doxopa_c 0.0001
h2o2_c 0.0197
h2o_c 8.637
rdmbzi_c 0.0001
Growth Rate
----------
0.06127911117806917


In [58]:
# Formate
with mcaps:
    open_bounds()
    mcaps.reactions.EX_no3_e.bounds = -1000, 1000
    mcaps.reactions.EX_ch4_e.bounds = 0, 0
    mcaps.reactions.SMMOi.bounds = 0, 1000
    mcaps.reactions.GLNS.bounds = 0, 1000
    mcaps.reactions.ALAD_L.bounds = 0, 1000
    mcaps.reactions.EX_co2_e.bounds = 0, 0
    mcaps.reactions.SK_for_c.bounds = -10, 0
    solution = pfba(mcaps)
    summary(mcaps, solution)
    
    # visualize_fluxes(mcaps, solution)
    

IN
----------
ca2_e 0.0043
cl_e 0.0131
cu2_e 0.0001
fe2_e 0.0004
for_c 10.0
h_e 10.5021
k_e 0.0108
mg2_e 0.0076
na1_e 0.0024
no3_e 0.5022
o2_e 1.8628
pi_e 0.0338
so4_e 0.0103
zn2_e 0.0001
OUT
----------
co2_c 7.9372
doxopa_c 0.0001
h2o2_c 0.0197
h2o_c 8.637
rdmbzi_c 0.0001
Growth Rate
----------
0.06127911117806891


In [66]:
from cobra.io import save_json_model

In [67]:
write_sbml_model(mcaps, '../ModelFiles/xml/iMcBath.xml')
save_json_model(mcaps, '../ModelFiles/json/iMcBath.json', pretty=True, sort=True)

In [70]:
mcaps.reactions.BIOMASS_Mcapsulatus.metabolites

{<Metabolite 2ddg6p_c at 0x113dc0470>: -3e-06,
 <Metabolite 2pg_c at 0x113df49b0>: -0.006,
 <Metabolite 6pgc_c at 0x113de87b8>: -0.00015,
 <Metabolite accoa_c at 0x113e2d550>: -0.0001,
 <Metabolite adocbl_c at 0x113dfc588>: -6e-08,
 <Metabolite ala__L_c at 0x113e13048>: -0.576,
 <Metabolite amet_c at 0x113e13f60>: -0.00022,
 <Metabolite arg__L_c at 0x113e5f7f0>: -0.254,
 <Metabolite asn__L_c at 0x113e5fe80>: -0.119,
 <Metabolite asp__L_c at 0x113e6fc50>: -0.468,
 <Metabolite atp_c at 0x113de18d0>: -23.11185589,
 <Metabolite ca2_c at 0x113e2ac50>: -0.06986377,
 <Metabolite cit_c at 0x113e88a58>: -0.001,
 <Metabolite cl_c at 0x113e96390>: -0.21438646,
 <Metabolite clpn_MC_c at 0x113e96b00>: -3.15904e-06,
 <Metabolite coa_c at 0x113e80208>: -0.00022,
 <Metabolite cobalt2_c at 0x113e801d0>: -5.769e-05,
 <Metabolite colipa_e at 0x113e65080>: -0.002,
 <Metabolite cpoa2h_c at 0x113e657f0>: -0.00186552,
 <Metabolite ctp_c at 0x113e8eac8>: -0.04425885,
 <Metabolite cu2_c at 0x113e8ed68>: -0.001